In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime


In [3]:
def get_release_date(game_name):

    try:

        url_params = game_name.replace(' ', '+')

        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en"
        }
        baseurl = 'https://en.wikipedia.org'
        url_search = '/w/index.php?search='
        
        log = 'Searching for ' + url_params + ' on Wikipedia...\n'
        html = requests.get(baseurl + url_search + url_params, headers=headers).text

        # converting text to bs4 object to parse the html tags
        soup = BeautifulSoup(html, features='html.parser')

        # if search is ambiguous, it will show a list of results, try with video game
        if not soup.find('th', class_='infobox-label', string='Release'):
            log += url_params + ' is ambiguous, searching for video game...' + '(full address: ' + baseurl + url_search + url_params + ')\n'
            links = soup.find_all('a', href=re.compile(r'video.?game', re.IGNORECASE))
            for link in links:
                log += f"Found link: {link.get('href')}\n"
            # take first link and update soup
            if links:
                log += f'Taking first link: {links[0].get("href")}\n'
                html = requests.get(baseurl + links[0].get('href'), headers=headers).text
                soup = BeautifulSoup(html, features='html.parser')  
            else:
                # try additional search with additional (video game) suffix            
                url_params_game = url_params + '(video+game)'
                log += f'No video game link found, trying search with (video game) suffix... (full address: {baseurl}{url_search}{url_params_game})\n'
                html = requests.get(baseurl + url_search + url_params_game, headers=headers).text
                soup = BeautifulSoup(html, features='html.parser')

        th = soup.find('th', class_='infobox-label', string='Release')
        tr = th.find_parent('tr')
        for td in tr.find_all('td'):
            text_str = td.get_text(separator=' ', strip=True)   
            log += 'Found release date string: ' + text_str + '\n'

        # Find first date pattern like "Month Day, Year"
        match = re.search(r'([A-Z][a-z]+)\s+(\d{1,2}),\s+(\d{4})', text_str)
        log += 'Extracted date string: ' + (match.group(0) if match else 'No date found') + '\n'
        if match: 
            date_str = match.group(0)
            date_obj = datetime.strptime(date_str, '%B %d, %Y')
            log += f"FOUND DATE: {date_obj.strftime('%d.%m.%Y')}\n"
            return date_obj.strftime('%d.%m.%Y'), log
        # ty to find date pattern like "Day Month Year"
        match = re.search(r'(\d{1,2})\s+([A-Z][a-z]+)\s+(\d{4})', text_str)
        log += 'Extracted date string: ' + (match.group(0) if match else 'No date found') + '\n'
        if match:
            date_str = match.group(0)
            date_obj = datetime.strptime(date_str, '%d %B %Y')
            log += f"FOUND DATE: {date_obj.strftime('%d.%m.%Y')}\n"
            print(log)
            return date_obj.strftime('%d.%m.%Y'), log
        return None, log
        
    except Exception as e:        
        log += f'An error occurred while trying to find the release date for {game_name}: {e}\n'
        print(log)
        return None, log

# get_release_date('The Legend of Zelda: Breath of the Wild')
# get_release_date('Doom')
get_release_date('The Pedestrian')

(None,
 'Searching for The+Pedestrian on Wikipedia...\nThe+Pedestrian is ambiguous, searching for video game...(full address: https://en.wikipedia.org/w/index.php?search=The+Pedestrian)\nFound link: /wiki/Fahrenheit_451_(video_game)\nTaking first link: /wiki/Fahrenheit_451_(video_game)\nFound release date string: 1984\nExtracted date string: No date found\nExtracted date string: No date found\n')

In [4]:
# connect to db
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv('../.env')
db_user = os.environ['DB_USER']
db_pass = os.environ['DB_PASS']
db_host = '192.168.178.124'
db_port = 5435
db_name = os.environ['DB_NAME']
print(f'Connecting to database {db_name} with user {db_user}')

engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

with engine.connect() as connection:
    result = connection.execute(text('SELECT game_id, name FROM games'))
    for row in result:
        print(f'Game ID: {row.game_id}, Name: {row.name}')
        


Connecting to database gaming with user flaskapp
Game ID: 108, Name: Age of Empires IV
Game ID: 43, Name: Ori and the Blind Forest: Definitive Edition
Game ID: 39, Name: Zup!
Game ID: 9, Name: Diabolo III
Game ID: 41, Name: Diabolo IV
Game ID: 59, Name: The Plucky Squire
Game ID: 36, Name: Diabolo II: Resurrected
Game ID: 44, Name: Ori and the Will of the Wisp
Game ID: 46, Name: Thronebreaker: The Witcher Tales
Game ID: 48, Name: Greak: Memories of Azur
Game ID: 31, Name: Cocoon
Game ID: 14, Name: Trine 3: The Artifacts of Power
Game ID: 53, Name: Creaks
Game ID: 52, Name: The Pedestrian
Game ID: 51, Name: DARQ
Game ID: 50, Name: Lost Words: Beyond the Page
Game ID: 49, Name: Sheepy: A Short Adventure
Game ID: 47, Name: White Shadows
Game ID: 63, Name: Candle
Game ID: 61, Name: Eiyuden Chronicle: Rising
Game ID: 60, Name: Tales of Kenzera: ZAU
Game ID: 58, Name: SteamWorld Dig
Game ID: 56, Name: The Case of the Golden Idol
Game ID: 55, Name: Saviorless
Game ID: 54, Name: Assemble with 

In [73]:
import time

# Create a new table with game_id and release_date
with engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS land_game_release_dates'))
    connection.commit()
    connection.execute(text('CREATE TABLE IF NOT EXISTS land_game_release_dates (game_id INTEGER PRIMARY KEY, name VARCHAR(255), release_date DATE, error BOOLEAN, log TEXT)'))
    connection.commit()
    result = connection.execute(text('SELECT game_id, name FROM games'))
    for row in result:
        time.sleep(1)  # Sleep for 1 second to avoid overwhelming Wikipedia with requests
        print('Processing name with game_id: ' + str(row[0]) + ': ' + row[1])
        release_date, log = get_release_date(row[1])
        if release_date:
            print('Inserting into database with release date: ' + release_date)
            connection.execute(text('INSERT INTO land_game_release_dates (game_id, name, release_date, error, log) VALUES (:game_id, :name, TO_DATE(:release_date, \'DD.MM.YYYY\'), FALSE, :log)'), {'game_id': row[0], 'name': row[1], 'release_date': release_date, 'log': log})  
        else:
            print('Inserting into database with error for game_id ' + str(row[0]) + ': ' + row[1])
            connection.execute(text('INSERT INTO land_game_release_dates (game_id, name, error, log) VALUES (:game_id, :name, TRUE, :log)'), {'game_id': row[0], 'name': row[1], 'log': log})
        connection.commit()


Processing name with game_id: 108: Age of Empires IV
Inserting into database with release date: 28.10.2021
Processing name with game_id: 43: Ori and the Blind Forest: Definitive Edition
Inserting into database with release date: 11.03.2015
Processing name with game_id: 39: Zup!
Searching for Zup! on Wikipedia...
Zup! is ambiguous, searching for video game...(full address: https://en.wikipedia.org/w/index.php?search=Zup!)
No video game link found, trying search with (video game) suffix... (full address: https://en.wikipedia.org/w/index.php?search=Zup!(video+game))
An error occurred while trying to find the release date for Zup!: 'NoneType' object has no attribute 'find_parent'

Inserting into database with error for game_id 39: Zup!
Processing name with game_id: 9: Diabolo III
Searching for Diabolo+III on Wikipedia...
Diabolo+III is ambiguous, searching for video game...(full address: https://en.wikipedia.org/w/index.php?search=Diabolo+III)
No video game link found, trying search with (

![alt text](<assets/Screenshot 2026-02-20 152805.png>)

In [5]:
# copy data from land_game_release_dates to games, update release_year in games table, only for rows where error is FALSE
with engine.connect() as connection:
    connection.execute(text('UPDATE games SET release_year = (SELECT EXTRACT(YEAR FROM release_date)::int FROM land_game_release_dates WHERE land_game_release_dates.game_id = games.game_id AND land_game_release_dates.error = FALSE)'))
    connection.commit()